In [2]:
import numpy as np

In [3]:
import requests
import os

In [4]:
import cv2
from PIL import Image

In [5]:
os.chdir('Documents/Projects/ImageRec')

In [6]:
import matplotlib.pyplot as plt

In [7]:
def img_save(query_term, offset):
    '''
    saves and processes images for specified query, creates folder in directory if there isnt one
    '''
    
    API_KEY = os.getenv('bing_search_api_key')
    URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)' + \
                 'Chrome/80.0.3987.87 Safari/537.36'
    
    # set path with query term as file name
    path = os.getcwd().replace('\\', '/') + '/' + str(query_term) + '/'
    
    if not os.path.exists(query_term):
        # if the directory does not exist, make one
        os.mkdir(query_term)
        
    headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
    params = {"q": query_term,
              "count": 150, 
              "offset": offset * 150}

    # Search Bing for images
    search = requests.get(URL, headers=headers, params=params)
    results = search.json()
    
    print('TOTAL ESTIMATED MATCHES: ' + str(results['totalEstimatedMatches']))

    # Save all of the resulting images from each page
    num = offset * 150
    for value in results['value']:

        print(str(num) + ' ' + value['contentUrl'], end='\r')
        
        try:
            image = requests.get(value["contentUrl"], timeout=30, headers={'User-Agent': USER_AGENT})
            
        except(requests.ConnectionError):
            print(str(num) + ' BAD CONNECTION', end='\r')
            continue
            
        except(requests.ReadTimeout):
            print(str(num) + ' TIMEOUT', end='\r')
            continue
        
        # Check the status of the request - If the image does not exist we will skip it
        try:
            image.raise_for_status()
            
            file = open(path + query_term + '_' + str(offset) + '_' + str(num) + '.png', 'wb')
            file.write(image.content)
            file.close()
            
        except(requests.HTTPError):
            print(str(num) + ' NOT FOUND', end='\r')
            
        num += 1

In [8]:
def img_format(folder):
    '''
    read in all images in given folder, format them, and put them into a master array
    return x_data array, y_data array
    '''
    
    x_data = np.array([])
    Y_data = np.array([])
    
    # Iterate through each file in the specified folder
    for file in os.listdir(folder):
        
        print(file, end='\r')
        
        # Read in the image
        img = np.array(Image.open(folder + '/' + file))
        
        # If the image is greyscale, discard it
        if len(img.shape) == 2:
            continue
        
        # If the image is 4 channel (RGBA), convert to 3 channel (RGB)
        if (len(img.shape) > 2) & (img.shape[2] == 4):
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        
        # Identify which sides need to be padded and by how much, to make the image square
        short = np.argmin(img.shape[:2])
        diff_1 = int(np.ceil(abs(img.shape[1] - img.shape[0])/2))
        diff_2 = int(np.floor(abs(img.shape[1] - img.shape[0])/2))
        
        # Set the desired padding on the short side, and apply
        width = [[0, 0], [0, 0], [0, 0]]
        width[short] = [diff_1, diff_2]
        img = np.pad(img, pad_width=width)
        
        # Resize square image to 100x100
        img = cv2.resize(img, (100, 100))
        
        # Reshape array to be appended to x_data array
        img = img.reshape(1, 100, 100, 3)
        
        # Put the formatted arrays into a master array of training data
        if np.array_equal(x_data, np.array([])):
            # This is the first one, start the array
            x_data = img
            
        else:
            # Append to full array
            x_data = np.concatenate((x_data, img), axis=0)
        
        # The class label will be same as the name of the folder
        Y_data = np.append(Y_data, folder)
    
    return x_data, Y_data

In [82]:
def img_process(img, nn_model):
    '''
    reads in a single image, proccesses it, and generates predictions from the neural network
    assumes incoming image is square
    returns prediction array
    '''
    
    # Resize to 100x100
    img = cv2.resize(img, (100, 100))
    
    # Reshape to NN friendly shape
    img = img.reshape(1, 100, 100, 3, 1)
    
    # Generate prediction array from given NN model
    pred = nn_model.predict(img.astype(float))
    
    return pred

In [84]:
img_process(img, nn_model)

array([[7.0406069e-15, 4.3367917e-14, 3.5749658e-16, 1.4462540e-07,
        9.9999988e-01]], dtype=float32)

Save the images from search queries

Format the images and save to arrays

In [9]:
x_cat, Y_cat = img_format('cat')

In [10]:
x_dog, Y_dog = img_format('dog')

In [11]:
x_snake, Y_snake = img_format('snake')

In [13]:
x_penguin, Y_penguin = img_format('penguin')

In [15]:
x_dolphin, Y_dolphin = img_format('dolphin')

In [18]:
x_train = np.concatenate((x_cat, x_dog, x_snake, x_penguin, x_dolphin))

In [19]:
x_train = x_train.reshape(len(x_train), 100, 100, 3, 1)

In [20]:
Y_train = np.concatenate((Y_cat, Y_dog, Y_snake, Y_penguin, Y_dolphin))

In [26]:
Y_train[Y_train == 'cat'] = 0
Y_train[Y_train == 'dog'] = 1
Y_train[Y_train == 'snake'] = 2
Y_train[Y_train == 'penguin'] = 3
Y_train[Y_train == 'dolphin'] = 4

In [38]:
Y_train = Y_train.astype(np.int)

Train NN

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [30]:
nn_model = Sequential()

In [31]:
nn_model.add(Dense(16, input_shape=x_train[0].shape, activation='relu'))
nn_model.add(Dense(16,  activation='relu'))
nn_model.add(Flatten())
nn_model.add(Dense(len(np.unique(Y_train)),  activation='softmax'))

In [32]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100, 100, 3, 16)   32        
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 3, 16)   272       
_________________________________________________________________
flatten (Flatten)            (None, 480000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2400005   
Total params: 2,400,309
Trainable params: 2,400,309
Non-trainable params: 0
_________________________________________________________________


In [39]:
nn_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

nn_model.fit(x=x_train,y=Y_train, epochs=10)

Train on 2451 samples
Epoch 1/10
2451/2451 [==============================] - 22s 9ms/sample - loss: 800.5372 - accuracy: 0.4109
Epoch 2/10
2451/2451 [==============================] - 21s 8ms/sample - loss: 79.8018 - accuracy: 0.5557
Epoch 3/10
2451/2451 [==============================] - 21s 9ms/sample - loss: 32.6090 - accuracy: 0.5916
Epoch 4/10
2451/2451 [==============================] - 22s 9ms/sample - loss: 7.6242 - accuracy: 0.6691
Epoch 5/10
2451/2451 [==============================] - 22s 9ms/sample - loss: 1.0181 - accuracy: 0.7609
Epoch 6/10
2451/2451 [==============================] - 23s 9ms/sample - loss: 0.6052 - accuracy: 0.7968
Epoch 7/10
2451/2451 [==============================] - 23s 9ms/sample - loss: 0.5990 - accuracy: 0.8001
Epoch 8/10
2451/2451 [==============================] - 25s 10ms/sample - loss: 0.5799 - accuracy: 0.8099
Epoch 9/10
2451/2451 [==============================] - 22s 9ms/sample - loss: 0.5776 - accuracy: 0.8070
Epoch 10/10
2451/2451 [=====

In [55]:
nn_model.predict(x_train[2001].reshape(1, 100, 100, 3, 1).astype(float))#.argmax()

array([[1.01032878e-04, 8.24486429e-04, 1.17715805e-04, 3.37377749e-02,
        9.65218961e-01]], dtype=float32)